In [4]:
import csv, sys
import os, glob
import string
from string import ascii_lowercase
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer, word_tokenize
import nltk
import pandas as pd


output_path = '/home/romina/Documents/work/Speeches_mining/outputs'
input_path = '/home/romina/Documents/work/speeches-segmented/greg-tables'

In [5]:
os.chdir(output_path)
output = open('output.csv', 'r')
data = pd.read_csv(output, encoding='utf-8')
text = pd.DataFrame(data[['text']])
data = list()
for index, content in text.iterrows():
    data.append(content['text'])

In [7]:
print(len(data))
text=list()
text.append(data[0])
text.append(data[1])


272217


In [8]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)
    ##setting up and extending stopwords
stop = stopwords.words('english')
stop.extend(
    ['nevertheless', 'would', 'nether''the', 'in', 'may', 'also', 'zero', 'one', 'two', 'three', 'four', 'five',
     'six', 'seven', 'eight', 'nine', 'ten', 'quot', 'across', 'among', 'beside', 'however', 'yet',
     'within', '1', '2', '3', '4', '5', '6', '7', '8', '9'] + list(ascii_lowercase))

tokens = []
for statement in text:
    # words = tokenizer.tokenize(statement.lower())
    words = word_tokenize(statement.lower())
    print(len(words))
    # print(words)
    words_clean = []
    for word in words:
        if word in stop:
            continue
        words_clean.append(stemmer.stem(word))
    tokens.append(words_clean)

my_ngrams = []
n = 3

for token_group in tokens:
    ngram = list(nltk.ngrams(token_group, n))
    my_ngrams.append(ngram)



1394
637


In [ ]:
from nltk.collocations import *
from nltk.probability import FreqDist

all_trigrams=list()
for ngram_group in my_ngrams:
    for trigram in ngram_group:
        all_trigrams.append(trigram)
total=len(all_trigrams)
cutoff = 0.00001 * total
fdist = FreqDist(all_trigrams)
# for k,v in fdist.items():
#     print(k,v)
print(cutoff)
selected_trigrmas={k: v for k, v in fdist.items() if v > cutoff}
print(selected_trigrmas)

0.011170000000000001
{}


In [60]:
ngrams_counter = dict()
total = 0
for token_group in my_ngrams:
    for token in token_group:
        total = total + 1
        my_key = tuple(token)
        if my_key in ngrams_counter:  
            ngrams_counter[my_key] = ngrams_counter[my_key] + 1
            continue
        ngrams_counter[my_key] = 1

max_occ = max(ngrams_counter.values())
cutoff = 0.00001 * total
ngrams_orderpos = list()

for key in ngrams_counter.keys():
    if ngrams_counter[key] < cutoff: continue
    ngrams_orderpos.append(key)


('speaker', 'move', 'suspend')
('move', 'suspend', 'rule')
('suspend', 'rule', 'pass')
('rule', 'pass', 'bill')
('pass', 'bill', 'hr')
('bill', 'hr', '3980')
('hr', '3980', 'provid')
('3980', 'provid', 'identifi')
('provid', 'identifi', 'elimin')
('identifi', 'elimin', 'redund')
('elimin', 'redund', 'report')
('redund', 'report', 'requir')
('report', 'requir', 'develop')
('requir', 'develop', 'meaning')
('develop', 'meaning', 'perform')
('meaning', 'perform', 'metric')
('perform', 'metric', 'homeland')
('metric', 'homeland', 'secur')
('homeland', 'secur', 'prepared')
('secur', 'prepared', 'grant')
('prepared', 'grant', 'purpos')
('grant', 'purpos', 'amend')
('purpos', 'amend', 'clerk')
('amend', 'clerk', 'read')
('clerk', 'read', 'titl')
('read', 'titl', 'bill')
('titl', 'bill', 'text')
('bill', 'text', 'bill')
('text', 'bill', 'follow')
('bill', 'follow', 'hr')
('follow', 'hr', '3980')
('hr', '3980', 'enact')
('3980', 'enact', 'senat')
('enact', 'senat', 'hous')
('senat', 'hous', 'rep

In [65]:

print(ngrams_orderpos)

[('speaker', 'move', 'suspend'), ('move', 'suspend', 'rule'), ('suspend', 'rule', 'pass'), ('rule', 'pass', 'bill'), ('pass', 'bill', 'hr'), ('bill', 'hr', '3980'), ('hr', '3980', 'provid'), ('3980', 'provid', 'identifi'), ('provid', 'identifi', 'elimin'), ('identifi', 'elimin', 'redund'), ('elimin', 'redund', 'report'), ('redund', 'report', 'requir'), ('report', 'requir', 'develop'), ('requir', 'develop', 'meaning'), ('develop', 'meaning', 'perform'), ('meaning', 'perform', 'metric'), ('perform', 'metric', 'homeland'), ('metric', 'homeland', 'secur'), ('homeland', 'secur', 'prepared'), ('secur', 'prepared', 'grant'), ('prepared', 'grant', 'purpos'), ('grant', 'purpos', 'amend'), ('purpos', 'amend', 'clerk'), ('amend', 'clerk', 'read'), ('clerk', 'read', 'titl'), ('read', 'titl', 'bill'), ('titl', 'bill', 'text'), ('bill', 'text', 'bill'), ('text', 'bill', 'follow'), ('bill', 'follow', 'hr'), ('follow', 'hr', '3980'), ('hr', '3980', 'enact'), ('3980', 'enact', 'senat'), ('enact', 'sena

In [68]:
#size(my_ngrms)=size(text)
D = np.zeros(shape=(len(my_ngrams), len(ngrams_orderpos)))

for index in range(0, len(my_ngrams)):
    for token in my_ngrams[index]:
        if token not in ngrams_orderpos: continue
        D[index, ngrams_orderpos.index(token)] = 1

np.savetxt("binary_matrix", D, delimiter=",")


In [67]:
print(D)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]


In [75]:
my_ngrams_counted = list()
for ngrams in my_ngrams:
    my_dict = dict()
    for ngram in ngrams:
        if ngram not in ngrams_orderpos: continue
        if ngram not in my_dict.keys():
            my_dict[ngram] = 1
            continue
        my_dict[ngram] = my_dict[ngram] + 1
    my_ngrams_counted.append(my_dict)

D_counted = np.zeros(shape=(len(my_ngrams), len(ngrams_orderpos)))
for index in range(0, len(my_ngrams_counted)):
    for key in my_ngrams_counted[index].keys():
        D_counted[index, ngrams_orderpos.index(key)] = my_ngrams_counted[index][key]
        
np.savetxt("counted_matrix", D, delimiter=",")


In [71]:
print(D_counted)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]]
